**Notes:** need to download csv, copy/change path and create df in step 1. change time.sleep() intervals to slow down/speed up process. Change the user agen in 'headers' to your user agent (search 'my user agent' in chrome). 

Be careful not to exceed Amazon's request rate limit! The first scrape takes about 30 mins at time.sleep(random.randint(1,4) and I the actualy product/price/review scrape can take up to an hour!

In [1]:
#import packages
import pandas as pd
import numpy as np
import requests
import time
import bs4
from bs4 import BeautifulSoup
import random

#download the spreadsheet that directs the bot at 'https://docs.google.com/spreadsheets/d/1j1zp8yHJbXMwSe58bW38MODuD9wOYDWfPhklsKTqjqI/edit?usp=sharing', its pretty straightforward
df = pd.read_csv('/content/Arms_AMZN - Sheet1 (1).csv')

#get page url's for all individual categories (note: URL IS VARIABLE, you should be able to use best sellers or another ranked AMZ category)
cat_names1 = {'names':[]}
cat_names2 = {'names':[]}
cat_names3 = {'names':[]}
for x in range(len(df)):
  try:
    num2 = df['second_arm'][x]
    num2 = int(num2) - 1
  except:
    print('num2 problem')
  try:
    num3 = df['third_arm'][x]
    num3 = int(num3) - 1
  except:
    print('num3 problem')
  try:
    url = 'https://www.amazon.com/gp/new-releases/'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
    cat_names = {'names':[]}
    html = requests.get(url,headers = headers).text
    soup = bs4.BeautifulSoup(html, "html.parser")
    for tag in soup.findAll("div", {"class": "_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-small__nleKL"}):
            cat_names['names'].append(tag.findNext("a"))
  except:
    print('arm 1 error')
  try:
    url = 'https://www.amazon.com'+ str(cat_names['names'][num2]).split('"')[1]
    html = requests.get(url,headers = headers).text
    soup = bs4.BeautifulSoup(html, "html.parser")
    for tag in soup.findAll("div", {"class": "_p13n-zg-nav-tree-all_style_zg-browse-group__88fbz"}):
            cat_names1['names'].append(tag.findAll("a"))
  except:
      print('arm 2 error')
  try:
    cat_names1['names']
    url2 = 'https://www.amazon.com'+ str(cat_names1['names'][num3]).split('"')[1]
    html = requests.get(url2,headers = headers).text
    soup = bs4.BeautifulSoup(html, "html.parser")
    for tag in soup.findAll("div", {"class": "_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-large__1z5B8"}):
            cat_names2['names'].append(tag.findAll("a"))
  except:
    print('arm 3 error')
  time.sleep(random.randint(1, 4))

#collect all collected links, droping any duplicates
all1 = {'link':[],'name':[]}
for x in range(len(cat_names1['names'])):
  for xx in range(len(cat_names1['names'][x])):
    all1['link'].append('https://www.amazon.com'+str(cat_names1['names'][x][xx]).split('"')[1])
    all1['name'].append(str(cat_names1['names'][x][xx]).split('"')[2])
print('length of secondary links collected'+str(len(pd.DataFrame(all1['link']).drop_duplicates())))
print('length of secondary names collected'+str(len(pd.DataFrame(all1['names']).drop_duplicates())))

all2 = {'link':[],'name':[]}
for x in range(len(cat_names2['names'])):
  for xx in range(len(cat_names2['names'][x])):
    all2['link'].append('https://www.amazon.com'+str(cat_names2['names'][x][xx]).split('"')[1])
    all2['name'].append(str(cat_names2['names'][x][xx]).split('"')[2])
print('length of primary links collected'+str(len(pd.DataFrame(all2['link']).drop_duplicates())))
print('length of primary names collected'+str(len(pd.DataFrame(all2['name']).drop_duplicates())))

#mor orgnaization... getting top then bottom level urls
aaa = pd.DataFrame()
aaa['link'] = pd.DataFrame(all2['link']).drop_duplicates()[0]
aaa['name'] = pd.DataFrame(all2['name']).drop_duplicates()[0]
aa = pd.DataFrame()
aa[['link','name']] = pd.merge(pd.DataFrame(all1['link']).reset_index(),pd.DataFrame(all1['name']).reset_index(), on = 'index').drop(columns = 'index').drop_duplicates()
aa = aa.append(aaa)
aa = aa.reset_index()
aa = aa.sort_values('index').drop(columns = 'index')

,second_arm,third_arm
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
549,36,11
550,36,12
551,36,13
552,36,14


In [127]:
#scraping the first + second image, rating + # of ratings, and price to observe day to day patterns
cat_names = {'imageLink':[],'imageLink1':[],'brand':[],'rating':[],'price':[]}
cat_names1 = {'imageLink':[],'imageLink1':[],'brand':[],'rating':[],'price':[]}

for x in range(len(aa['names'])):
  try:
    url = aa['link'][x]
    url2 = str(aa['link'][x]).split('ref=')[0]+'ref='+str(aa['link'][x]).split('ref=')[1].split("_1")[0]+'_2?ie=UTF8&pg=2'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
    html = requests.get(url,headers = headers).text
    soup = bs4.BeautifulSoup(html, "html.parser")
    for tag in soup.findAll("div", {"class": "a-section a-spacing-mini _p13n-zg-list-grid-desktop_maskStyle_noop__3Xbw5"}):
            tt = {'imageLink':[]}
            tt['imageLink'].append(tag.findAll("img"))
            for x in range(len(tt['imageLink'])):
              cat_names['brand'].append(str(tt['imageLink'][x]).split('"')[1])
              cat_names['imageLink'].append(str(tt['imageLink'][x]).split('"')[5])
              cat_names['imageLink1'].append(str(tt['imageLink'][x]).split('"')[7])
            
    for tag in soup.findAll("div", {"class": "a-icon-row"}):
            cat_names['rating'].append(tag.findNext("span").text)
    for tag in soup.findAll("span", {"class": "a-size-base a-color-price"}):
            cat_names['price'].append(tag.findNext("span",{"class":"_p13n-zg-list-grid-desktop_price_p13n-sc-price__3mJ9Z"}).text)
  except:
    print('first 50, error with link: '+ str(url))

  time.sleep(random.randint(1,2))
#-------------------------------------------------------------------------------------------------------------
  try:
    html1 = requests.get(url2,headers = headers).text
    soup1 = bs4.BeautifulSoup(html1, "html.parser")
    for tag in soup1.findAll("div", {"class": "a-section a-spacing-mini _p13n-zg-list-grid-desktop_maskStyle_noop__3Xbw5"}):
            tt = {'imageLink':[]}
            tt['imageLink'].append(tag.findAll("img"))
            for x in range(len(tt['imageLink'])):
              cat_names1['brand'].append(str(tt['imageLink'][x]).split('"')[1])
              cat_names1['imageLink'].append(str(tt['imageLink'][x]).split('"')[5])
              cat_names1['imageLink1'].append(str(tt['imageLink'][x]).split('"')[7])
            
    for tag in soup.findAll("div", {"class": "a-icon-row"}):
            cat_names1['rating'].append(tag.findNext("span").text)
    for tag in soup.findAll("span", {"class": "a-size-base a-color-price"}):
            cat_names1['price'].append(tag.findNext("span",{"class":"_p13n-zg-list-grid-desktop_price_p13n-sc-price__3mJ9Z"}).text)

    print(len(cat_names1['imageLink']))
    print(len(cat_names1['imageLink1']))
    print(len(cat_names1['brand']))
    print(len(cat_names1['rating']))
    print(len(cat_names1['price']))

  except:
    print('second 50, error with link: '+ url2)
  
  time.sleep(random.randint(1,2))


In [ ]:
#next steps: delete duplicate products + organize data, note: selenium needed to access 40/100 products per category --> bs4 doesnt scroll scrape and my comp doesn't work with selenium!